# Práctica 2

Procesamiento de Lenguaje Natural
Facultad de Ingeniería, UNAM

González Flores Andrés

## Instrucciones

A partir del corpus proporcionado ("corpusML.txt") realizar un modelo del lenguaje neuronal con base en la arquitectura propuesta por Bengio (2003).

Síganse los siguientes pasos:

1. Limpiar los textos y aplicar stemming a las palabras.
2. Insertar símbolos de inicio y final de cadena.
3. Obtener los bigramas que aparecen en el texto (indexar numéricamente).
4. Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparámetros. Tomar de 100 unidades para la primera capa oculta (capa lineal) y 300 para la segunda capa oculta (capa con tanh).
5. Obtener las matrices $A$ y $\Pi$ a partir de las salidas de la red neuornal (probabilidad Softmax).
6. Evaluar el modelo (con Entropía).
7. Calcular la proabilidad de las siguientes oraciones:
    - Nos bañamos con agua caliente
    - El animalito le olía la cabeza
    - Pascuala ordeñaba las vacas

## Desarrollo

In [1]:
# Importar módulos
import re
from nltk import SnowballStemmer
from collections import Counter, defaultdict
from itertools import chain

In [2]:
# Definición de constantes
stemmer_esp = SnowballStemmer('spanish')
CORPUS_PATH = './Data/corpusML.txt'
BOS = '<BOS>'
EOS = '<EOS>'

In [3]:
# def vocab():
#     # Un defaultdict es una subclase de dict, el primer argumento
#     vocab = defaultdict() 
#     # El método default_factory sirve para asignar un nuevo valor por defecto
#     # en caso de no encontrar el indice dado
#     vocab.default_factory = lambda: len(vocab)
#     return vocab

# def text2number(corpus, vocab):
#     for doc in corpus:
#         yield [vocab[w] for w in doc]

### Paso 1

Limpiar los textos y aplicar stemming a las palabras.

In [4]:
with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
    corpus = [
        list(map(lambda token: stemmer_esp.stem(token), 
            re.findall('[a-zA-zñáéíóúü]+', frase.lower())
        ))
        for frase in f
    ]
    for stems in corpus[:5]:
        print(stems) # Muestro 5 ejemplos

['comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con', 'chicot']
['mis', 'patron', 'me', 'peg', 'porqu', 'no', 'me', 'quer', 'apur', 'porqu', 'era', 'floj']
['por', 'eso', 'me', 'hab', 'peg']
['cuand', 'me', 'peg', 'ya', 'entonc', 'me', 'quit']
['pues', 'entonc', 'no', 'quis', 'trabaj']


### Paso 2

Insertar símbolos de inicio y final de cadena.

In [5]:
corpus = list(map(lambda stems: [BOS, *stems, EOS], corpus))
for stems in corpus[:5]:
    print(stems)

['<BOS>', 'comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con', 'chicot', '<EOS>']
['<BOS>', 'mis', 'patron', 'me', 'peg', 'porqu', 'no', 'me', 'quer', 'apur', 'porqu', 'era', 'floj', '<EOS>']
['<BOS>', 'por', 'eso', 'me', 'hab', 'peg', '<EOS>']
['<BOS>', 'cuand', 'me', 'peg', 'ya', 'entonc', 'me', 'quit', '<EOS>']
['<BOS>', 'pues', 'entonc', 'no', 'quis', 'trabaj', '<EOS>']


### Paso 3

Obtener los bigramas que aparecen en el texto (indexar numéricamente).

In [6]:
# Indexo numéricamente los stems
vocab = defaultdict() 
# El método default_factory sirve para asignar un nuevo valor por defecto
# en caso de no encontrar el indice dado
vocab.default_factory = lambda: len(vocab)

# Indexo numéricamente las palabras en los documentos
corpus_ids = [[vocab[w] for w in doc] for doc in corpus]

print('Ejemplos de palabras en el vocabulario')
for palabra, i in list(vocab.items())[:10]:
    print(f'  {palabra} : {i}')

print('\nEjemplos de oraciones indexadas numéricamente')
for doc in corpus_ids[:6]:
    print(' ', doc)

Ejemplos de palabras en el vocabulario
  <BOS> : 0
  comenc : 1
  a : 2
  trabaj : 3
  y : 4
  me : 5
  peg : 6
  maltrat : 7
  con : 8
  chicot : 9

Ejemplos de oraciones indexadas numéricamente
  [0, 1, 2, 3, 4, 5, 6, 5, 7, 8, 9, 10]
  [0, 11, 12, 5, 6, 13, 14, 5, 15, 16, 13, 17, 18, 10]
  [0, 19, 20, 5, 21, 6, 10]
  [0, 22, 5, 6, 23, 24, 5, 25, 10]
  [0, 26, 24, 14, 27, 3, 10]
  [0, 23, 14, 27, 28, 29, 5, 6, 10]


```doc[:-1]``` toma todos las cadenas de un documento (frase) excepto la última

```doc[1:]``` toma todos las cadenas de un documento (frase) excepto la primera

Con zip, uno una cadena de la primer lista con una de la segunda en tuplas (bigramas)

In [7]:
bigramas = [bi for doc in corpus for bi in zip(doc[:-1], doc[1:])]

bigramas[:20] # Ejemplos con cadenas

[('<BOS>', 'comenc'),
 ('comenc', 'a'),
 ('a', 'trabaj'),
 ('trabaj', 'y'),
 ('y', 'me'),
 ('me', 'peg'),
 ('peg', 'me'),
 ('me', 'maltrat'),
 ('maltrat', 'con'),
 ('con', 'chicot'),
 ('chicot', '<EOS>'),
 ('<BOS>', 'mis'),
 ('mis', 'patron'),
 ('patron', 'me'),
 ('me', 'peg'),
 ('peg', 'porqu'),
 ('porqu', 'no'),
 ('no', 'me'),
 ('me', 'quer'),
 ('quer', 'apur')]

In [8]:
bigramas_ids = [bi for doc in corpus_ids for bi in zip(doc[:-1], doc[1:])]
bigramas_ids[:20] # Ejemplos de bigramas indexados numéricamente

[(0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 5),
 (5, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (0, 11),
 (11, 12),
 (12, 5),
 (5, 6),
 (6, 13),
 (13, 14),
 (14, 5),
 (5, 15),
 (15, 16)]